# Supplemental Code - COGS 118A Final Project

## Evaluation Metrics

In [2]:
# Finding TP, FP, FN, and TN using a confusion matric

import numpy as np

def compute_confusion_matrix_per_patient(labels, predictions, threshold, seqLength, snooze_length = 0):   
    # Fill-in confusion matrix.
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    snooze = 0
    
    for t in range(seqLength):       
        # Only check labels and predictions when not snoozed.
        if snooze == 0:           
            label = labels[t] == 1
            prediction = predictions[t] >= threshold 
            if label and prediction:
                tp += 1
            elif not label and prediction:
                fp += 1
            elif label and not prediction:
                fn += 1
            elif not label and not prediction:
                tn += 1

            if prediction:
                snooze = snooze_length
                
        else:
            snooze -= 1
            
    return tp, fp, fn, tn

In [3]:
# Computing the AUC using the predictions and the confusion matrices for each patient.
def compute_snoozed_auc(labels, predictions, predictionsConcatenate, seqLength, snooze_length = 0, num_thresholds = 0):
    # Check input for errors.
    if len(labels) != len(predictions):
        raise Exception('Numbers of labels and predictions must be equal.')
    if any(len(X) != len(Y) for X, Y in zip(labels, predictions)):
        raise Exception('Numbers of labels and predictions must be equal for all samples.')
    if any(not (x == 0 or x == 1) for X in labels for x in X):
        raise Exception('Labels must be binary.')
    if any(not (0 <= y <= 1) for Y in predictions for y in Y):
        raise Exception('Predictions must be between 0 and 1, inclusive.')

    # Find prediction thresholds.
    sorted_predictions = [np.unique(Y) for Y in predictions]

    if not num_thresholds:
        thresholds = np.unique(np.concatenate(sorted_predictions))[::-1]
    else:
        percentiles = np.linspace(0, 100, num_thresholds)
        thresholds = np.unique(np.percentile(np.concatenate(sorted_predictions), percentiles, interpolation='nearest'))[::-1]

    if thresholds[0] != 1:
        thresholds = np.insert(thresholds, 0, 1)
    if thresholds[-1] == 0:
        thresholds = thresholds[:-1]
    num_thresholds = len(thresholds)

    num_patients = labels.shape[0]

    # Populate confusion matrix for each prediction threshold.
    sample_tp = np.zeros(num_patients)
    sample_fp = np.zeros(num_patients)
    sample_fn = np.zeros(num_patients)
    sample_tn = np.zeros(num_patients)
    sample_indices = -np.ones(num_patients, dtype=np.int)

    tp = np.zeros(num_thresholds)
    fp = np.zeros(num_thresholds)
    fn = np.zeros(num_thresholds)
    tn = np.zeros(num_thresholds)

    for i in range(num_thresholds):
        for j in range(num_patients):
            k = np.searchsorted(sorted_predictions[j], thresholds[i])
            if k != sample_indices[j]:
                a, b, c, d = compute_confusion_matrix_per_patient(labels[j], predictions[j], thresholds[i], seqLength[j], snooze_length)
                sample_tp[j] = a
                sample_fp[j] = b
                sample_fn[j] = c
                sample_tn[j] = d
                sample_indices[j] = k

            tp[i] += sample_tp[j]
            fp[i] += sample_fp[j]
            fn[i] += sample_fn[j]
            tn[i] += sample_tn[j]

    # Summarize confusion matrix.
    tpr = np.zeros(num_thresholds)
    tnr = np.zeros(num_thresholds)
    ppv = np.zeros(num_thresholds)

    for i in range(num_thresholds):
        if tp[i] + fn[i]:
            tpr[i] = tp[i] / (tp[i] + fn[i])
        else:
            tpr[i] = 1
        if fp[i] + tn[i]:
            tnr[i] = tn[i] / (fp[i] + tn[i])
        else:
            tpr[i] = 0
        if tp[i] + fp[i]:
            ppv[i] = tp[i] / (tp[i] + fp[i])
        else:
            ppv[i] = 1

    # Compute AUROC as the area under a piecewise linear function of TPR
    # (x-axis) and FPR (y-axis) and AUPRC as the area under a piecewise linear
    # function of recall (x-axis) and precision (y-axis).
    auroc = 0
    auprc = 0
    for i in range(num_thresholds - 1):
        auroc += 0.5 * (tpr[i + 1] - tpr[i]) * (tnr[i + 1] + tnr[i])
        auprc += 0.5 * (tpr[i + 1] - tpr[i]) * (ppv[i + 1] + ppv[i])


    return auroc, auprc

In [4]:
# Computing the confusion matrix and PPV within a given time horizon
def compute_confusion_matrix_per_patient_ppvModified(labels, predictions, threshold, seqLength, false_positive_prediction_horizon, snooze_length = 0):   
    # Fill-in confusion matrix.
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    tp_modified = 0
    fp_modified = 0

    snooze = 0
    
    for t in range(seqLength):       
        # Only check labels and predictions when not snoozed.
        if snooze == 0:           
            label = labels[t] == 1
            prediction = predictions[t] >= threshold 
           
            # Is there a positive label within the horizon?
            label_horizon = np.any(labels[t:min(seqLength, t+false_positive_prediction_horizon+1)])

            if label_horizon and prediction:
                tp += 1
            elif not label_horizon and prediction:
                fp += 1
            elif label and not prediction:
                fn += 1
            elif not label and not prediction:
                tn += 1

            if label_horizon and prediction:
                tp_modified += 1
            elif not label_horizon and prediction:
                fp_modified += 1

            if prediction:
                snooze = snooze_length
                
        else:
            snooze -= 1
            
    return tp, fp, fn, tn, tp_modified, fp_modified

In [5]:
# Computing all the evaluation metrics (calling the other functions above) 
def compute_ppvModified_snoozed_auc(labels, predictions, seqLength, false_positive_prediction_horizon=24, snooze_length = 0, num_thresholds = 0):
    # Check input for errors.
    if len(labels) != len(predictions):
        raise Exception('Numbers of labels and predictions must be equal.')
    if any(len(X) != len(Y) for X, Y in zip(labels, predictions)):
        raise Exception('Numbers of labels and predictions must be equal for all samples.')
    if any(not (x == 0 or x == 1) for X in labels for x in X):
        raise Exception('Labels must be binary.')
    if any(not (0 <= y <= 1) for Y in predictions for y in Y):
        raise Exception('Predictions must be between 0 and 1, inclusive.')

    # Find prediction thresholds.
    #num_cohorts = len(labels)
    sorted_predictions = [np.unique(Y) for Y in predictions]

    if not num_thresholds:
        thresholds = np.unique(np.concatenate(sorted_predictions))[::-1]
    else:
        percentiles = np.linspace(0, 100, num_thresholds)
        thresholds = np.unique(np.percentile(np.concatenate(sorted_predictions), percentiles, interpolation='nearest'))[::-1]

    if thresholds[0] != 1:
        thresholds = np.insert(thresholds, 0, 1)
    if thresholds[-1] == 0:
        thresholds = thresholds[:-1]
    num_thresholds = len(thresholds)
    
    # Find prediction thresholds.
    #thresholds = np.unique(predictionsConcatenate)[::-1]
    #if thresholds[0] != 1:
    #    thresholds = np.insert(thresholds, 0, 1)
    #if thresholds[-1] == 0:
    #    thresholds = thresholds[:-1]
    #num_thresholds = len(thresholds)

    num_patients = labels.shape[0]

    # Populate confusion matrix for each prediction threshold.
    sample_tp = np.zeros(num_patients)
    sample_fp = np.zeros(num_patients)
    sample_fn = np.zeros(num_patients)
    sample_tn = np.zeros(num_patients)
    sample_indices = -np.ones(num_patients, dtype=np.int)

    # Initialize arrays for computing modified PPV
    sample_tp_modified = np.zeros(num_patients)
    sample_fp_modified = np.zeros(num_patients)

    tp_modified = np.zeros(num_thresholds)
    fp_modified = np.zeros(num_thresholds)

    tp = np.zeros(num_thresholds)
    fp = np.zeros(num_thresholds)
    fn = np.zeros(num_thresholds)
    tn = np.zeros(num_thresholds)

    for i in range(num_thresholds):
        for j in range(num_patients):
            k = np.searchsorted(sorted_predictions[j], thresholds[i])
            if k != sample_indices[j]:
                a, b, c, d, e, f = compute_confusion_matrix_per_patient_ppvModified(labels[j], predictions[j], thresholds[i], seqLength[j], false_positive_prediction_horizon, snooze_length)
                sample_tp[j] = a
                sample_fp[j] = b
                sample_fn[j] = c
                sample_tn[j] = d
                sample_indices[j] = k

                sample_tp_modified[j] = e
                sample_fp_modified[j] = f

            tp[i] += sample_tp[j]
            fp[i] += sample_fp[j]
            fn[i] += sample_fn[j]
            tn[i] += sample_tn[j]

            tp_modified[i] += sample_tp_modified[j]
            fp_modified[i] += sample_fp_modified[j]

    # Summarize confusion matrix.
    tpr = np.zeros(num_thresholds)
    tnr = np.zeros(num_thresholds)
    ppv = np.zeros(num_thresholds)

    ppv_modified = np.zeros(num_thresholds) 

    for i in range(num_thresholds):
        if tp[i] + fn[i]:
            tpr[i] = tp[i] / (tp[i] + fn[i])
        else:
            tpr[i] = 1
        if fp[i] + tn[i]:
            tnr[i] = tn[i] / (fp[i] + tn[i])
        else:
            tpr[i] = 0
        if tp[i] + fp[i]:
            ppv[i] = tp[i] / (tp[i] + fp[i])
        else:
            ppv[i] = 1

        if tp_modified[i] + fp_modified[i]:
            ppv_modified[i] = tp_modified[i] / (tp_modified[i] + fp_modified[i])
        else:
            ppv_modified[i] = 1

    # Compute AUROC as the area under a piecewise linear function of TPR
    # (x-axis) and FPR (y-axis) and AUPRC as the area under a piecewise linear
    # function of recall (x-axis) and precision (y-axis).
    auroc = 0
    auprc = 0
    auprc_modified = 0
    for i in range(num_thresholds - 1):
        auroc += 0.5 * (tpr[i + 1] - tpr[i]) * (tnr[i + 1] + tnr[i])
        auprc += (tpr[i + 1] - tpr[i]) * (ppv[i + 1])
        auprc_modified += (tpr[i + 1] - tpr[i]) * (ppv_modified[i + 1])

#     for i in range(num_thresholds):
#         print(i, thresholds[i], tp[i], fp[i], fn[i], tn[i])

    return auroc, auprc, auprc_modified, tpr, fp_modified, ppv_modified, tnr, thresholds